<a href="https://colab.research.google.com/github/thanhvietnguyen/Titanic-ML-project/blob/master/app/api/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib
# Using the BytesIO you create a file object out of the response that you get from GitHub
from io import BytesIO
import requests

app = FastAPI()

# Load mô hình
mLink = 'https://github.com/thanhvietnguyen/Titanic-ML-project/blob/master/models/best_model.pkl?raw=true'
try:
    print("Đang tải mô hình từ GitHub...")
    response = requests.get(mLink)
    response.raise_for_status()  # Gây lỗi nếu không=200
    mfile = BytesIO(response.content)
    model = joblib.load(mfile)
    print("Mô hình đã tải thành công.")
except Exception as e:
    print("Lỗi khi tải mô hình:", str(e))

class Passenger(BaseModel):
    Pclass: int
    Sex: str
    Age: float
    SibSp: int
    Parch: int
    Fare: float
    Embarked: str

@app.get("/")
def read_root():
    return {"message": "Titanic Prediction API is online!"}

@app.post("/predict")
def predict(passenger: Passenger):
  if model is None:
    return {"Error": "Model chưa được tải thành công"}

  try:
    family_size = passenger.SibSp + passenger.Parch + 1
    input_df = pd.DataFrame([{
            "Age": passenger.Age,
            "Fare": passenger.Fare,
            "FamilySize": family_size,
            "Sex": passenger.Sex,
            "Embarked": passenger.Embarked,
            "Pclass": passenger.Pclass
        }])
    print("Dữ liệu đầu vào:\n", input_df)
    pred = model.predict(input_df)[0]
    prob = model.predict_proba(input_df)[0][1]

    return {
        "prediction": int(pred),
        "probability_survival": round(prob, 4)
    }
  except Exception as e:
        print("Lỗi dự đoán:", str(e))
        return {"Error": f"Lỗi khi xử lý dự đoán: {str(e)}"}

Đang tải mô hình từ GitHub...
Mô hình đã tải thành công.
